# One iteration of Arnoldi's algorithm

In [6]:
import numpy as np

We import the full arnoldi algorithm to compare the results with the one iteration algorithm

In [7]:
from preconditioner import iter_precondition

def arnoldi(A, b, r0, m, precondition = False, M = None, tol = 1e-12):
    """
    This function computes an orthonormal basis V_m = {v_1,...,v_{m+1}} of 
    K_{m+1}(A, r^{(0)}) = span{r^{(0)}, Ar^{(0)}, ..., A^{m}r^{(0)}}.

    Input parameters:
    -----------------
      A: array_like
          An (n x n) array.
      
      b: array_like
          Initial vector of length n
      
      m: int
          One less than the dimension of the Krylov subspace. Must be > 0.
      
      x0: array_like 
          Initial approximation of the solution (length n)
      
      tol: 
          Tolerance for convergence

    Output:
    -------
      Q: numpy.array 
          n x m array, the columns are an orthonormal basis of the Krylov subspace.
      
      H: numpy.array
          An (m + 1) x m array. It is the matrix A on basis Q. It is upper Hessenberg.
    """
    
    # TODO: Now we calculate precondition time at each iteration. The idea is that arnoldi algorithm only provides one 
    # - iteration at a time so we don't build every time the whole basis from scratch in the GMRES algorithm
    # - torch?
    # - instead of A, callable function that calculates Ax for any input vector x?
    # Check inputs
    n = A.shape[0]
    assert A.shape == (n, n) and b.shape == (n,) and x0.shape == (n,), "Matrix and vector dimensions don not match"
    assert isinstance(m, int) and m > 0, "m must be a positive integer"
    
    m = min(m, n)
    
    # Initialize matrices
    V = np.zeros((n, m + 1))
    H = np.zeros((m + 1, m))
    
    # Normalize input vector and use for Krylov vector
    
    beta = np.linalg.norm(r0)
    V[:, 0] = r0 / beta

    for k in range(1, m + 1):
        # Generate a new candidate vector
        w, _= iter_precondition(A, M, V, k, precondition)
        
        # Orthogonalization
        for j in range(k):
            H[j, k - 1] = V[:, j] @ w
            w -= H[j, k - 1] * V[:, j]
        
        H[k, k - 1] = np.linalg.norm(w)

        # Check convergence
        if H[k, k - 1] <= tol:
            print(f"Converged in {k} iterations.")
            return V, H
        
        # Normalize and store the new basis vector
        V[:, k] = w / H[k, k - 1]
    
    return V, H

In [8]:
def arnoldi_one_iter(A, V, k, precondition=False, M=None, tol = 1e-12):
    """
    Computes one iteration of Arnoldi iteration given the iteration index k

    :param precondition:
    :param A:
    :param V:
    :param k:
    :return:
    """

    # inialize k + 1 nonzero elements of H along column k. k starts at 0...
    h_k = np.zeros((k + 2, ))

    w, iter_precondition_time = iter_precondition(A, M, V, k + 1, precondition)
    
    # Calculate first k elements of the kth Hessenberg column
    for j in range(k + 1): # Here k is from 0 to 
        h_k[j] = w @ V[:, j]
        w = w - h_k[j] * V[:, j]

    # h_k[k + 1] = sp_norm(w)
    h_k[k + 1] = np.linalg.norm(w)

    # check if 0
    if h_k[k + 1] == 0:
        # None for v to check in gmres (early termination with EXACT SOLUTION)
        return h_k, None, 0
    else:
        # find the new orthogonal vector in the basis of the Krylov subspace
        # assert h_k[k + 1] != 0
        v_new = w / h_k[k + 1]

    # v_new = w / h_k[k + 1]

    return h_k, v_new, iter_precondition_time

# Unit tests

### Small matrix just to verify the algorithm is working

In [9]:
from preconditioner import PreconditionEnum, initial_precondition

In [13]:
n = 10
m = 1
rand = np.random.RandomState(0)

A = rand.rand(n, n)
b = rand.rand(n)
x0 = np.zeros(n)

r0 = b - A @ x0
precondition = PreconditionEnum.JACOBI

M, _, _ = initial_precondition(A, precondition, r0) # r0 is not used here but in the PRECONDITIONED GMRES

# V, H = arnoldi(A, b, x0, m, precondition = precondition, M = M)
V, H = arnoldi(A, b, r0, m, precondition = None, M = None)

n, _ = A.shape

V = np.zeros((n, 1))
r0 = b - A @ x0
beta = np.linalg.norm(r0)
V[:, 0] = r0 / beta
V = np.concatenate((V, np.zeros((n, 1))), axis=1)

H = np.zeros((n + 1, 1))
H = np.concatenate((H, np.zeros((n + 1, 1))), axis=1)
k = 0

H[:(k + 2), k], v_new, iter_precondition_time = arnoldi_one_iter(A, V, k, precondition=False, M=None)

V[:, k + 1] = v_new

(array([[0.33772937],
        [0.13453437],
        [0.36631832],
        [0.47942078],
        [0.12394393],
        [0.28707658],
        [0.29499125],
        [0.28513066],
        [0.11115282],
        [0.47471743]]),
 array([[3.92980991],
        [1.98254355]]))

In [65]:
V[:, : k + 1] , H[: k + 2, : k +1]

array([[ 0.33772937,  0.17493401],
       [ 0.13453437,  0.62463971],
       [ 0.36631832,  0.15463533],
       [ 0.47942078, -0.14437439],
       [ 0.12394393,  0.28209878],
       [ 0.28707658,  0.01984779],
       [ 0.29499125, -0.12367891],
       [ 0.28513066,  0.11598928],
       [ 0.11115282,  0.4612879 ],
       [ 0.47471743, -0.46147275]])

In [44]:
H[:(k + 1)]

array([[3.92980991, 0.        ]])

In [50]:
from numpy.linalg import qr
Q, R = qr(H[:(k + 1)], mode = 'complete')
Q[0][:-1]

array([], dtype=float64)